In [1]:
# modelling potential:

# df_activity: medium but dataset too small
# df_weight: low
# df_sec: aggregate to 1/5/15 min interval
# df_min: intensity?
# df_hr: lifestyle trends

### Contents

- [EDA](#EDA)

### Header

In [2]:
# user configuration

In [3]:
# import libraries

# maths
import numpy as np
import pandas as pd

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# modelling
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix,accuracy_score,r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.utils import resample, shuffle
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier

# Others
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

### Import Data

In [4]:
# file paths

input_path = '../data/1_input/'
clean_path = '../data/2_clean/'
preprocess_path = '../data/3_preprocess/'
output_path = '../data/4_output/'

folders = ['Fitabase Data 3.12.16-4.11.16','Fitabase Data 4.12.16-5.12.16']

In [5]:
# import clean data

df_activity = pd.read_csv(clean_path + 'activity.csv')
df_weight = pd.read_csv(clean_path + 'weight.csv')

df_sec = pd.read_csv(clean_path + 'seconds.csv')
df_min = pd.read_csv(clean_path + 'minutes.csv')
df_hr = pd.read_csv(clean_path + 'hours.csv')

### Functions

### Data Overview

In [6]:
print(df_activity.shape)
df_activity.head()

(457, 17)


,id,timestamp,dayweek,year,month,day,steps,calories,dist_total,dist_act_high,dist_act_med,dist_act_low,dist_act_sede,mins_act_high,mins_act_med,mins_act_low,mins_act_sede
0,1,2016-03-25,Friday,2016,3,25,11004,1819,7.10,2.57,0.46,4.07,0.0,33,12,205,804
1,1,2016-03-26,Saturday,2016,3,26,17609,2154,11.56,6.92,0.73,3.91,0.0,89,17,274,588
2,1,2016-03-27,Sunday,2016,3,27,12736,1944,8.53,4.66,0.16,3.71,0.0,56,5,268,605
3,1,2016-03-28,Monday,2016,3,28,13231,1932,8.93,3.19,0.79,4.95,0.0,39,20,224,1080
4,1,2016-03-29,Tuesday,2016,3,29,12041,1886,7.86,2.16,1.09,4.61,0.0,28,28,243,763


In [7]:
df_activity.columns

Index(['id', 'timestamp', 'dayweek', 'year', 'month', 'day', 'steps',
       'calories', 'dist_total', 'dist_act_high', 'dist_act_med',
       'dist_act_low', 'dist_act_sede', 'mins_act_high', 'mins_act_med',
       'mins_act_low', 'mins_act_sede'],
      dtype='object')

### Preprocess Data

In [8]:
mask_steps = df_activity['steps'] >= 1000
df_activity_2 = df_activity[mask_steps]

In [9]:
# df_activity['mins_total'] = df_activity['mins_act_high'] + df_activity['mins_act_med'] + df_activity['mins_act_low'] + df_activity['mins_act_sede']

# df_activity['mins_pct_high'] = df_activity['mins_act_high'] / df_activity['mins_total']
# df_activity['mins_pct_med'] = df_activity['mins_act_med'] / df_activity['mins_total']
# df_activity['mins_pct_low'] = df_activity['mins_act_low'] / df_activity['mins_total']
# df_activity['mins_pct_sede'] = df_activity['mins_act_sede'] / df_activity['mins_total']

In [10]:
# df_activity.head()

In [11]:
cols = ['timestamp','year','month', 'day']
df_activity_2.drop(columns=cols,inplace=True)

In [12]:
df_activity_2 = pd.get_dummies(df_activity_2,columns=['dayweek'],drop_first=True)

print(df_activity_2.shape)
df_activity_2.head()

(370, 18)


,id,steps,calories,dist_total,dist_act_high,dist_act_med,dist_act_low,dist_act_sede,mins_act_high,mins_act_med,mins_act_low,mins_act_sede,dayweek_Monday,dayweek_Saturday,dayweek_Sunday,dayweek_Thursday,dayweek_Tuesday,dayweek_Wednesday
0,1,11004,1819,7.10,2.57,0.46,4.07,0.0,33,12,205,804,0,0,0,0,0,0
1,1,17609,2154,11.56,6.92,0.73,3.91,0.0,89,17,274,588,0,1,0,0,0,0
2,1,12736,1944,8.53,4.66,0.16,3.71,0.0,56,5,268,605,0,0,1,0,0,0
3,1,13231,1932,8.93,3.19,0.79,4.95,0.0,39,20,224,1080,1,0,0,0,0,0
4,1,12041,1886,7.86,2.16,1.09,4.61,0.0,28,28,243,763,0,0,0,0,1,0


### Create Features and Target

In [14]:
target = 'id'
features = [x for x in df_activity_2.columns if x != target]

X = df_activity_2[features].values
y = df_activity_2[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3,stratify=y,random_state=3050)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(259, 17)
(111, 17)
(259,)
(111,)


In [16]:
#df_activity_2.iloc[:,:12]
#df_activity_2.iloc[:,-6:]
#X_train[0,:11]

In [17]:
ss = StandardScaler()
ss.fit(X_train[:,:11])

# X_train = ss.transform(X_train)
# X_test = ss.transform(X_test)
X_train_scale = ss.transform(X_train[:,:11])
X_test_scale = ss.transform(X_test[:,:11])

X_train[:,:11] = X_train_scale
X_test[:,:11] = X_test_scale

#X_train_dum = X_train[:,-6:]
#X_test_dum = X_test[:,-6:]

In [19]:
print(X_train.shape)
print(X_test.shape)
print(X_train_scale.shape)
print(X_test_scale.shape)
# print(X_train_dum.shape)
# print(X_test_dum.shape)

(259, 17)
(111, 17)
(259, 11)
(111, 11)


### Logistic Regression Model

In [20]:
# init model

logreg = LogisticRegression()

In [21]:
# perform cross validation

score = cross_val_score(logreg,X,y,cv=5)
mse = cross_val_score(logreg,X,y,cv=5,scoring='neg_mean_squared_error')
r2 = cross_val_score(logreg,X,y,cv=5,scoring='r2')

print('score:',score.mean(),score)
print('mse:',mse.mean(),mse)
print('r2:',r2.mean(),r2)

score: 0.5277344877344877 [0.46590909 0.5952381  0.51388889 0.53030303 0.53333333]
mse: -87.52849206349205 [-77.85227273 -84.42857143 -91.38888889 -93.77272727 -90.2       ]
r2: 0.16956774888889606 [0.25414687 0.20773687 0.15487983 0.08757595 0.14349922]


In [24]:
# fit model

logreg.fit(X_train, y_train)

# score model
print("train r2:",logreg.score(X_train, y_train))
print("test r2:",logreg.score(X_test, y_test))

# validate model
y_pred = logreg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("rmse:",rmse)

train r2: 0.6525096525096525
test r2: 0.36036036036036034
rmse: 11.114068675445663


In [25]:
cols = ['y_test']
df_result = pd.DataFrame(y_test,columns=cols)

df_result['y_pred'] = y_pred
df_result['result'] = False

for idx,row in df_result.iterrows():  
    
    if row['y_test'] == row['y_pred']:
        df_result.at[idx,'result'] = True
        
df_result
#df_result.to_csv('df_result.csv')

### KNN Model

In [29]:
knn = KNeighborsClassifier(n_neighbors=2)

#n_neighbors=2
#n_neighbors=3
#n_neighbors=4
n_neighbors=5

In [30]:
# perform cross validation

score = cross_val_score(knn,X,y,cv=5)
mse = cross_val_score(knn,X,y,cv=5,scoring='neg_mean_squared_error')
r2 = cross_val_score(knn,X,y,cv=5,scoring='r2')

print('score:',score.mean(),score)
print('mse:',mse.mean(),mse)
print('r2:',r2.mean(),r2)

score: 0.3377417027417028 [0.29545455 0.27380952 0.40277778 0.33333333 0.38333333]
mse: -139.86233766233767 [-142.89772727 -127.35714286 -122.45833333 -152.01515152 -154.58333333]
r2: -0.32870698780478474 [-0.36901227 -0.19509743 -0.13243534 -0.47913242 -0.46785748]


In [31]:
# fit model

knn.fit(X_train, y_train)

# score model
print("train r2:",knn.score(X_train, y_train))
print("test r2:",knn.score(X_test, y_test))

# validate model
y_pred = knn.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("rmse:",rmse)

train r2: 0.6872586872586872
test r2: 0.35135135135135137
rmse: 12.165525060596439
